In [1]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
import matplotlib.pyplot as plt

In [31]:
def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    centerX = int((xA + xB)/2)
    centerY = int((yA + yB)/2)
    
    return iou, (centerX, centerY)

In [4]:
def convertBack(x, y, w, h):
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

def cvDrawBoxes(detections, img):
#     print('detections length : ', len(detections))
    pt1 = (0, 0)
    pt2 = (0, 0)
    area = 0
    
    xmin = 0
    ymin = 0
    xmax = 0
    ymax = 0
    
    for detection in detections:
        if detection[0].decode() != 'car' and detection[0].decode() != 'bus' and detection[0].decode() != 'truck':
            continue

        x, y, w, h = detection[2][0],\
            detection[2][1],\
            detection[2][2],\
            detection[2][3]
        xmin, ymin, xmax, ymax = convertBack( float(x), float(y), float(w), float(h))
        pt1 = (xmin, ymin)
        pt2 = (xmax, ymax)
        cv2.rectangle(img, pt1, pt2, (0, 255, 0), 3)
#         cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), -1)
        cv2.putText(img,
                    detection[0].decode() +
                    " [" + str(round(detection[1] * 100, 2)) + "]",
                    (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    [0, 255, 0], 2)
        area = (ymax-ymin) * (xmax-xmin)
    return img, area, (xmin, ymin, xmax, ymax)

outputWidth = 1920
outputHeight = 1080
netMain = None
metaMain = None
altNames = None 

In [30]:
global metaMain, netMain, altNames
configPath = "./cfg/yolov3-spp.cfg"
weightPath = "./yolov3-spp.weights"
metaPath = "./cfg/coco.data"
if not os.path.exists(configPath):
    raise ValueError("Invalid config path `" +
                     os.path.abspath(configPath)+"`")
if not os.path.exists(weightPath):
    raise ValueError("Invalid weight path `" +
                     os.path.abspath(weightPath)+"`")
if not os.path.exists(metaPath):
    raise ValueError("Invalid data file path `" +
                     os.path.abspath(metaPath)+"`")
if netMain is None:
    netMain = darknet.load_net_custom(configPath.encode(
        "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
if metaMain is None:
    metaMain = darknet.load_meta(metaPath.encode("ascii"))
if altNames is None:
    try:
        with open(metaPath) as metaFH:
            metaContents = metaFH.read()
            import re
            match = re.search("names *= *(.*)$", metaContents,
                              re.IGNORECASE | re.MULTILINE)
            if match:
                result = match.group(1)
            else:
                result = None
            try:
                if os.path.exists(result):
                    with open(result) as namesFH:
                        namesList = namesFH.read().strip().split("\n")
                        altNames = [x.strip() for x in namesList]
            except TypeError:
                pass
    except Exception:
        pass
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("./photi-vision-data/parking03_edit.mp4")
cap.set(3, outputWidth)
cap.set(4, outputHeight)
out = cv2.VideoWriter(
    "./photi-vision-data/output-test-3.mp4", cv2.VideoWriter_fourcc(*"MP4V"), 30.0,
    (outputWidth, outputHeight))
print("Starting the YOLO loop...")

#darknet.set_gpu(0)
# Create an image we reuse for each detect
darknet_image = darknet.make_image(outputWidth,
                                   outputHeight,3)

point1 = (0, 0)
point2 = (0, 0)
A = (point1, point2)
AREA = 1
X0, Y0, X1, Y1 = [0,0,0,0]
rectangleA = (0, 0, 0, 0) #기준이 되는 rectangle 선언

temp_images = []
temp_rectangles = []
while True:
    prev_time = time.time()
    ret, frame_read = cap.read()

    if ret == False:
        break

    frame_rgb = cv2.cvtColor(frame_read, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb,
                               (outputWidth,
                                outputHeight),
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image,frame_resized.tobytes())

    detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.50)
    image, area, rectangleB = cvDrawBoxes(detections, frame_resized)

    try:
        iou, center = bb_intersection_over_union(rectangleA, rectangleB)
        print('::::', iou, '::::', center, '::::::', rectangleB , '::::::', rectangleA)
    except ZeroDivisionError:
        print("ZeroDivision")
    if iou < 0.5 and iou > 0:
        cx = center[0]
        cy = center[1]
        cv2.line(image, (cx, cy),(cx, cy), (255,0,0), 50)
    elif iou == 0:
        cv2.line(image, (cx, cy),(cx, cy), (0,0,255), 50)

    rectangleA = rectangleB
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    out.write(image)
    cv2.waitKey(3)
    
cap.release()
out.release()

Starting the YOLO loop...
:::: 0.0 :::: (479, 251) :::::: (959, 502, 989, 531) :::::: (0, 0, 0, 0)
:::: 0.9375 :::: (974, 516) :::::: (959, 501, 989, 532) :::::: (959, 502, 989, 531)
:::: 1.0 :::: (974, 516) :::::: (959, 501, 989, 532) :::::: (959, 501, 989, 532)
:::: 0.96875 :::: (974, 516) :::::: (958, 501, 989, 532) :::::: (959, 501, 989, 532)
:::: 0.9696969696969697 :::: (973, 516) :::::: (957, 501, 989, 532) :::::: (958, 501, 989, 532)
:::: 0.9411764705882353 :::: (973, 516) :::::: (958, 501, 990, 532) :::::: (957, 501, 989, 532)
:::: 0.9696969696969697 :::: (973, 516) :::::: (958, 501, 989, 532) :::::: (958, 501, 990, 532)
:::: 0.0 :::: (994, 519) :::::: (999, 506, 1044, 550) :::::: (958, 501, 989, 532)
:::: 0.9565217391304348 :::: (1021, 528) :::::: (999, 507, 1043, 550) :::::: (999, 506, 1044, 550)
:::: 0.9565002471576866 :::: (1021, 528) :::::: (999, 507, 1044, 549) :::::: (999, 507, 1043, 550)
:::: 0.0 :::: (1439, 556) :::::: (1834, 564, 1915, 884) :::::: (999, 507, 1044, 549